In [ ]:
#libraries
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_rows", None, "mode.chained_assignment", None)



In [ ]:
# read csv
care_site = pd.read_csv('../datasets/CARE_SITE.csv')
cohort_filter = pd.read_csv('../datasets/COHORT_FILTER.csv')
concept_class= pd.read_csv("../datasets/CONCEPT_CLASS.csv")
concept = pd.read_csv("../datasets/CONCEPT.csv")
condition_occurrence = pd.read_csv("../datasets/CONDITION_OCCURRENCE.csv")
domain = pd.read_csv('../datasets/DOMAIN.csv')
drug_exposure = pd.read_csv('../datasets/DRUG_EXPOSURE.csv')
location = pd.read_csv('../datasets/LOCATION.csv')
measurement = pd.read_csv('../datasets/MEASUREMENT.csv', low_memory=False)
observation = pd.read_csv('../datasets/OBSERVATION.csv')
person = pd.read_csv('../datasets/PERSON.csv')
procedure_occurrence = pd.read_csv('../datasets/PROCEDURE_OCCURRENCE.csv')
provider = pd.read_csv('../datasets/PROVIDER.csv')
vocabulary = pd.read_csv('../datasets/VOCABULARY.csv')

In [ ]:
#concept lookup in concept df
concept_lookup = {c.concept_id: c.concept_name for c in concept.itertuples()}

In [ ]:
#function to make labels
def make_labels(df):
    for c in df.columns:
        if 'concept_id' in c:
            df[c.replace('_id', '_label')] = df[c].map(concept_lookup)
        if 'concept_id' in c or 'source' in c or len(df[df[c].notna()])==0:
            df = df.drop(c, axis=1)
    return df

In [ ]:
#
care_site_labelled = make_labels(care_site)
cohort_filter_labelled = make_labels(cohort_filter)
condition_occurrence_labelled = make_labels(condition_occurrence)
domain_labelled = make_labels(domain)
drug_exposure_labelled = make_labels(drug_exposure)
location_labelled = make_labels(location)
measurement_labelled = make_labels(measurement)
observation_labelled = make_labels(observation)
person_labelled = make_labels(person)
procedure_occurrence_labelled = make_labels(procedure_occurrence)
provider_labelled = make_labels(provider)
vocabulary_labelled = make_labels(vocabulary)

In [ ]:
person_code=

In [ ]:
cond_indiv = condition_occurrence_labelled.loc[condition_occurrence_labelled['person_id']==person_code]
drug_indiv = drug_exposure_labelled.loc[drug_exposure_labelled['person_id']==person_code]
meas_indiv = measurement_labelled.loc[measurement_labelled['person_id']==person_code]

display(cond_indiv.head())
display(drug_indiv.head())
display(meas_indiv.head())
display(meas_indiv[meas_indiv['measurement_concept_label']=='Body weight'])
display(meas_indiv[meas_indiv['measurement_concept_label']=='Weight change'])

In [ ]:
obs_indiv=observation_labelled.loc[observation_labelled['person_id']==person_code]
proc_indiv=procedure_occurrence_labelled.loc[procedure_occurrence_labelled['person_id']==person_code]
display(obs_indiv.head())
display(proc_indiv.head())

In [ ]:
cond_indiv.rename(columns={'condition_start_date':'start', 'condition_end_date':'end'}, inplace=True)
cond_indiv['content'] = cond_indiv['condition_type_concept_label'] + ':<br>' + cond_indiv['condition_concept_label']
cond_indiv['style'] = 'background-color: #f1aeb5;' #bootstrap $red-200
cond_indiv.head()

In [ ]:
drug_indiv.rename(columns={'drug_exposure_start_date':'start', 'drug_exposure_end_date':'end'}, inplace=True)
drug_indiv['content'] = drug_indiv['drug_concept_label'] + ', ' + drug_indiv['route_concept_label']
drug_indiv['style'] = 'background-color: #a3cfbb;' #bootstrap $green-200
drug_indiv.head()

In [ ]:
meas_indiv['measurement_concept_label'].value_counts()

In [ ]:
meas_weight = meas_indiv.loc[meas_indiv['measurement_concept_label'].isin(['Body weight','Weight change'])]
#Merge weight measurements with weight differences from last measurement
meas_weight['value_as_number']=meas_weight['value_as_number'].apply(str)
meas_grouped = meas_weight.groupby(['measurement_date'])['value_as_number'].transform(lambda x: ' kg,<br>Weight change: '.join(x))
meas_weight['value_as_number'] = meas_grouped
meas_weight['content'] = 'Body weight: ' + meas_weight['value_as_number']
#Rename columns
meas_weight.rename(columns={'measurement_date':'start'}, inplace=True)
meas_weight['style'] = 'background-color: #9ec5fe;' #bootstrap $blue-200
meas_weight.head()

In [ ]:
proc_indiv.rename(columns={'procedure_date':'start'}, inplace=True)
proc_indiv['content'] = proc_indiv['procedure_type_concept_label'] + ':<br>' + proc_indiv['procedure_concept_label']
proc_indiv['style'] = 'background-color: #c5b3e6;' #bootstrap $purple-200
proc_indiv.head()

In [ ]:
frames = [cond_indiv, drug_indiv, meas_weight, proc_indiv]
tl_events_merged = pd.concat(frames, ignore_index=True, sort=False)
tl_events_merged.head()

In [ ]:
tl_event_list = tl_events_merged[['start','end','content','style']]
tl_event_list.head()

In [ ]:
tl_event_list['end'] = tl_event_list['end'].fillna(tl_event_list['start'])
tl_event_list.head()